In [2]:
!pip install --upgrade torchtext

     |████████████████████████████████| 81kB 378kB/s eta 0:00:01
     |████████████████████████████████| 1.0MB 1.6MB/s eta 0:00:01


In [83]:
!pip install tqdm

In [1]:
!pipenv shell

Shell for /home/belerico/.local/share/virtualenvs/aml-project-EQl709OG already activated.
No action taken to avoid nested environments.


In [4]:
import re
import json
import torch
import string
from nltk import word_tokenize
from nltk.corpus import stopwords

english_stopwords = set(stopwords.words("english"))
non_alphanum_regex = re.compile(r'\W+')

def preprocess(doc, method='nltk', dataset=True):
    if method == 'spacy':
        tokens = " ".join(
            [
                token.lower_
                for token in doc
                if token
                and not (token.lower_ == "null" or token.is_stop or token.is_punct)
            ]
        )
    elif method == 'nltk':
        # doc = non_alphanum_regex.sub(' ', doc).lower()
        tokens = [
                token
                for token in word_tokenize(doc.lower())
                if not (token == "null" or token in english_stopwords or token in string.punctuation)
            ]
    elif method == 'keras':
        tokens = " ".join(
            [
                token
                for token in text_to_word_sequence(doc)
                if not (token == "null" or token in english_stopwords or token in string.punctuation)
            ]
        )
    if dataset or tokens != "":
        return tokens


def parse_content_line(x, attributes=None, label=True):
    if attributes is None:
        attributes = ["title_left", "title_right"]
    item = json.loads(x)
    elements = [item[attr] if item[attr] is not None else '' for attr in attributes]
    if label:
        elements.append(int(item["label"]))
    item = np.array(elements)
    return item[np.newaxis, :]

In [5]:
from torchtext.data import Field

TEXT = Field(sequential=True, tokenize=preprocess, lower=True, fix_length=20, batch_first=True)
LABEL = Field(sequential=False, use_vocab=False, is_target=True, batch_first=True, dtype=torch.float)

In [6]:
import numpy as np
from pandas import pandas

contents = []
for i, x in enumerate(open('./dataset/computers/train/computers_splitted_train_medium.json', "r").readlines()):
    try:
        item = parse_content_line(x, attributes=None, label=True)
        contents.append(item)
    except:
        print("Lost data at line {}".format(i))

contents = np.concatenate(contents, axis=0)
train = pandas.DataFrame(data=contents, columns=['title_left', 'title_right', 'label'])

In [7]:
import numpy as np
from pandas import pandas

contents = []
for i, x in enumerate(open('./dataset/computers/valid/computers_splitted_valid_medium.json', "r").readlines()):
    try:
        item = parse_content_line(x, attributes=None, label=True)
        contents.append(item)
    except:
        print("Lost data at line {}".format(i))

contents = np.concatenate(contents, axis=0)
valid = pandas.DataFrame(data=contents, columns=['title_left', 'title_right', 'label'])

In [8]:
import numpy as np
from pandas import pandas

contents = []
for i, x in enumerate(open('./dataset/computers/test/computers_gs.json', "r").readlines()):
    try:
        item = parse_content_line(x, attributes=None, label=True)
        contents.append(item)
    except:
        print("Lost data at line {}".format(i))

contents = np.concatenate(contents, axis=0)
test = pandas.DataFrame(data=contents, columns=['title_left', 'title_right', 'label'])

In [9]:
from torchtext.data import Field, Dataset, Example
import pandas as pd

class DataFrameDataset(Dataset):
    """Class for using pandas DataFrames as a datasource"""
    def __init__(self, examples, fields, filter_pred=None):
        """
        Create a dataset from a pandas dataframe of examples and Fields
        Arguments:
            examples pd.DataFrame: DataFrame of examples
            fields {str: Field}: The Fields to use in this tuple. The
                string is a field name, and the Field is the associated field.
            filter_pred (callable or None): use only exanples for which
                filter_pred(example) is true, or use all examples if None.
                Default is None
        """
        self.examples = examples.apply(SeriesExample.fromSeries, args=(fields,), axis=1).tolist()
        if filter_pred is not None:
            self.examples = filter(filter_pred, self.examples)
        self.fields = dict(fields)
        # Unpack field tuples
        for n, f in list(self.fields.items()):
            if isinstance(n, tuple):
                self.fields.update(zip(n, f))
                del self.fields[n]
        
                
class SeriesExample(Example):
    """Class to convert a pandas Series to an Example"""
    @classmethod
    def fromSeries(cls, data, fields):
        return cls.fromdict(data.to_dict(), fields)
    
    @classmethod
    def fromdict(cls, data, fields):
        ex = cls()
        for key, field in fields.items():
            if key not in data:
                raise ValueError("Specified key {} was not found in "
                "the input data".format(key))
            if field is not None:
                setattr(ex, key, field.preprocess(data[key]))
            else:
                setattr(ex, key, data[key])
        return ex

In [10]:
fields = {"title_left": TEXT, 'title_right': TEXT, 'label': LABEL}
train_ds = DataFrameDataset(train, fields)
valid_ds = DataFrameDataset(valid, fields)
test_ds = DataFrameDataset(test, fields)

In [11]:
print(train_ds[0].title_left)
print(valid_ds[0].title_left)
print(test_ds[0].title_left)

['asus', 'prime', 'x299', 'deluxe', 'prijzen', 'tweakers']
['495906', 'b21', 'hp', 'x5560', '2', '80ghz', 'ml350', 'g6', 'new', 'wholesale', 'price']
['417772', 'b21', 'hp', 'xeon', '5130', '2', '0ghz', 'dl140', 'g3', 'new', 'wholesale', 'price']


In [14]:
TEXT.build_vocab(train_ds, valid_ds, test_ds)

In [16]:
import gensim
from torchtext.vocab import Vectors

model = gensim.models.KeyedVectors.load_word2vec_format('./dataset/embeddings/w2v/w2v_title_300Epochs_1MinCount_9ContextWindow_150d.bin', binary=True)
# needed vectors not in binary form
vectors = Vectors(name='new_w2v_title_300Epochs_1MinCount_9ContextWindow_150d.bin', cache='./dataset/embeddings/w2v') # model_name + path = path_to_embeddings_file
TEXT.vocab.set_vectors(vectors.stoi, vectors.vectors, vectors.dim)

In [17]:
from torchtext.data import Iterator, BucketIterator

train_iter, val_iter, test_iter = BucketIterator.splits(
     (train_ds, valid_ds, test_ds), # we pass in the datasets we want the iterator to draw data from
     batch_sizes=(32, 64, 64),
     device=torch.device('cpu'), # if you want to use the GPU, specify the GPU number here
     sort_key=lambda x: min(max(len(x.title_left), len(x.title_right)), 20), # the BucketIterator needs to be told what function it should use to group the data.
     sort_within_batch=True,
     shuffle=True,
     repeat=False # we pass repeat=False because we want to wrap this Iterator layer.
)

In [18]:
class BatchWrapper:
    def __init__(self, dl, x_vars, y_var):
        self.dl, self.x_vars, self.y_var = dl, x_vars, y_var # we pass in the list of attributes for x 

    def __iter__(self):
        for batch in self.dl:
            left = getattr(batch, self.x_vars[0]) # we assume only one input in this wrapper
            right = getattr(batch, self.x_vars[1]) # we assume only one input in this wrapper
            y = torch.Tensor(list(map(float, getattr(batch, self.y_var))))

            yield (left, right, y)

    def __len__(self):
        return len(self.dl)

train_dl = BatchWrapper(train_iter, ['title_left', 'title_right'], 'label')
valid_dl = BatchWrapper(val_iter, ['title_left', 'title_right'], 'label')
test_dl = BatchWrapper(test_iter, ['title_left', 'title_right'], 'label')

In [19]:
next(train_dl.__iter__())

(tensor([[  62,  236, 1533,  114,    6,   28,   37,   74,   69,  216,  185,   56,
           844,  555,   24,    8,    1,    1,    1,    1],
         [  47,   19,  140,  360,  137,   77,   55,   86,   58,   38, 1920, 1899,
          1276,   43,    1,    1,    1,    1,    1,    1],
         [ 720,  428,   12,   11,   27,  795,    4,  388,   56,  185,  260,   31,
            96, 1164,   84,    1,    1,    1,    1,    1],
         [ 449,  115,   72,   71,  371,  348, 1701,  545,   83, 2561,  873, 1387,
          1339,   25,   18,    1,    1,    1,    1,    1],
         [  13,   70,  106, 1110,  237,   77,   33,   86,   58,   38,  860,    1,
             1,    1,    1,    1,    1,    1,    1,    1],
         [  13,   46,  131,   70,  475,   19,  513,   33,  914,   82,  111,   58,
           921,  329,  155,    1,    1,    1,    1,    1],
         [  53,   55,  245, 1140,  412,  144,    8,    1,    1,    1,    1,    1,
             1,    1,    1,    1,    1,    1,    1,    1],
         [  4

In [41]:
import gensim
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

class SimpleLSTMBaseline(nn.Module):
    def __init__(self, hidden_dim, emb_dim=150):
        super().__init__() # don't forget to call this!
        # model = gensim.models.KeyedVectors.load_word2vec_format('./dataset/embeddings/w2v/w2v_title_300Epochs_1MinCount_9ContextWindow_150d.bin', binary=True)
        # weights = torch.FloatTensor(model.vectors)
        self.embedding = nn.Embedding.from_pretrained(TEXT.vocab.vectors, padding_idx=TEXT.vocab.stoi[TEXT.pad_token])
        self.encoder_left = nn.LSTM(emb_dim, hidden_dim, num_layers=1, bidirectional=True, batch_first=True)
        self.encoder_right = nn.LSTM(emb_dim, hidden_dim, num_layers=1, bidirectional=True, batch_first=True)
        self.conv1 = nn.Conv2d(1, 8, 2)
        self.batch_norm1 = nn.BatchNorm2d(8)
        self.max_pool1 = nn.MaxPool2d(2)
        self.mlp1 = nn.Linear(648, 64)
        self.mlp2 = nn.Linear(64, 32)
        self.mlp3 = nn.Linear(32, 16)
        self.out = nn.Linear(16, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, seq):
        output_left, _ = self.encoder_left(self.embedding(seq[0]))
        output_right, _ = self.encoder_right(self.embedding(seq[1]))
        similarity = torch.matmul(output_left, torch.transpose(output_right, 1, 2))
        similarity = torch.unsqueeze(similarity, 1)
        x = self.conv1(similarity)
        x = F.relu(x)
        # x = self.batch_norm1(x)
        x = self.max_pool1(x)
        x = torch.flatten(x, start_dim=1)
        x = self.mlp1(x)
        x = F.relu(x)
        x = self.mlp2(x)
        x = F.relu(x)
        x = self.mlp3(x)
        x = F.relu(x)
        x = self.out(x)
        x = F.relu(x)
        x = self.sigmoid(x)
        return x

In [42]:
model = SimpleLSTMBaseline(hidden_dim=150, emb_dim=150)

In [ ]:
import tqdm
import functools
import operator
from sklearn.metrics import classification_report

opt = optim.Adam(model.parameters(), lr=1e-4)
loss_func = nn.BCELoss()

epochs = 10

y_true = [v[2] for v in valid_dl]
y_true = functools.reduce(operator.iconcat, y_true, [])

for epoch in range(1, epochs + 1):
    running_loss = 0.0
    running_corrects = 0
    model.train() # turn on training mode
    for left, right, y in train_dl: # thanks to our wrapper, we can intuitively iterate over our data!
        opt.zero_grad()
        preds = model([left, right])
        loss = loss_func(preds, torch.unsqueeze(y, 1))
        loss.backward()
        opt.step()
        running_loss += loss.data.item()
        
    epoch_loss = running_loss / len(train_dl)

    # calculate the validation loss for this epoch
    val_loss = 0.0
    model.eval() # turn on evaluation mode
    predictions = []
    for left, right, y in valid_dl:
        preds = model([left, right])
        predictions.extend(preds.data > .5)
        loss = loss_func(preds, torch.unsqueeze(y, 1))
        val_loss += loss.data.item()
    
    results = classification_report(y_true, predictions, digits=4, output_dict=True)
    val_loss /= len(valid_dl)
    print('Epoch: {}, Training Loss: {:.4f}, Validation Loss: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1: {:.4f}'.format(epoch, epoch_loss, val_loss, results['weighted avg']['precision'], results['weighted avg']['recall'], results['weighted avg']['f1-score']))

In [24]:
y_true = [v[2] for v in test_dl]
y_true = functools.reduce(operator.iconcat, y_true, [])
predictions = []
model.eval() # turn on evaluation mode
for left, right, y in test_dl:
    preds = model([left, right])
    predictions.extend(preds.data > .5)
print(classification_report(y_true, predictions))

              precision    recall  f1-score   support

         0.0       0.75      0.90      0.82       800
         1.0       0.41      0.19      0.26       300

    accuracy                           0.70      1100
   macro avg       0.58      0.54      0.54      1100
weighted avg       0.66      0.70      0.66      1100

